# MSDS 7331 Mini - Project 1  

## SVM and Logistic Modeling

Professor: Dr. Jake Drew  
Team: Steven Hayden, Josephine MacDaniel, Korey MacVittie, Afreen Siddiqui, Eduardo Cantu

## Models

In [1]:
#load Libraries
import pandas as pd
import numpy as np
import seaborn as seaborn
import matplotlib.pyplot as plt
import matplotlib as mp1
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


#Load Data
Accident_df = pd.read_csv('https://raw.githubusercontent.com/ecantu75/DataMining_Lab1/master/Data/accident_2016.csv',low_memory=False)

### Data Cleaning

In [4]:
# Displays the record count of non-null Values per attribute and their data type. 
Accident_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15855 entries, 1 to 34438
Data columns (total 75 columns):
Unnamed: 0                                                     15855 non-null int64
state_number                                                   15855 non-null int64
state_name                                                     15855 non-null object
consecutive_number                                             15855 non-null int64
number_of_vehicle_forms_submitted_all                          15855 non-null int64
number_of_motor_vehicles_in_transport_mvit                     15855 non-null int64
number_of_parked_working_vehicles                              15855 non-null int64
number_of_forms_submitted_for_persons_not_in_motor_vehicles    15855 non-null int64
number_of_persons_not_in_motor_vehicles_in_transport_mvit      15855 non-null int64
number_of_persons_in_motor_vehicles_in_transport_mvit          15855 non-null int64
number_of_forms_submitted_for_persons_in_motor_veh

In [11]:
Accident_df['day_of_crash'].max()

31

In [2]:
#Remove any recoreds without  time informaiton for arival and crash time
count_no_rec= Accident_df[(Accident_df['hour_of_crash']>24) | (Accident_df['hour_of_arrival_at_scene']>24)]
Accident_df = Accident_df[(Accident_df['hour_of_crash']<=24) & (Accident_df['hour_of_arrival_at_scene']<=24)]
print(count_no_rec.consecutive_number.count(), 'records were removed due to missing time data.') 


18584 records were removed due to missing time data.


In [3]:
# Add a new Feature. This is the ratio between the number of fatalities and the people involved in the accident.
Accident_df['Fatalities_ratio'] = Accident_df.number_of_fatalities/(Accident_df.number_of_persons_not_in_motor_vehicles_in_transport_mvit + Accident_df.number_of_persons_in_motor_vehicles_in_transport_mvit)

#Converts Hour and min to datetime type
Accident_df.hour_of_crash = pd.to_timedelta(Accident_df.hour_of_crash,unit ='h')
Accident_df.minute_of_crash= pd.to_timedelta(Accident_df.minute_of_crash,unit ='m')
Accident_df.hour_of_arrival_at_scene = pd.to_timedelta(Accident_df.hour_of_arrival_at_scene,unit ='h')
Accident_df.minute_of_arrival_at_scene = pd.to_timedelta(Accident_df.minute_of_arrival_at_scene,unit ='m')

#concatenates Hour and Minutes together 
Accident_df['Crash_Time'] = Accident_df['hour_of_crash'] + Accident_df['minute_of_crash'] 
Accident_df['Arrival_Time'] = Accident_df['hour_of_arrival_at_scene'] + Accident_df['minute_of_arrival_at_scene']
#creates a response_time variable from the two fields above
Accident_df['Response_Time'] = Accident_df['Arrival_Time'] - Accident_df['Crash_Time']

#gut check of calculation 
Accident_df[['hour_of_crash','minute_of_crash','Crash_Time','hour_of_arrival_at_scene','minute_of_arrival_at_scene','Arrival_Time','Response_Time']].head()



,hour_of_crash,minute_of_crash,Crash_Time,hour_of_arrival_at_scene,minute_of_arrival_at_scene,Arrival_Time,Response_Time
1,02:00:00,00:33:00,02:33:00,03:00:00,00:25:00,03:25:00,00:52:00
12,01:00:00,00:39:00,01:39:00,01:00:00,00:46:00,01:46:00,00:07:00
33,18:00:00,00:58:00,18:58:00,18:00:00,00:31:00,18:31:00,-1 days +23:33:00
40,20:00:00,00:00:00,20:00:00,19:00:00,00:38:00,19:38:00,-1 days +23:38:00
54,19:00:00,00:26:00,19:26:00,19:00:00,00:39:00,19:39:00,00:13:00


### Logistic Regression Model

### Support Vector Machine Model

## Model Discussion

## Logistic Regression Weight Interpretation

## Insights based on the Classification Task